# **IPL Data Analysis Using Python Pandas**

**Little detail about IPL (Indian Premier League).
The Indian Premier League (IPL), officially Vivo Indian Premier League for sponsorship reasons, is a professional Twenty20 cricket league in India contested during April and May of every year by teams representing Indian cities and some states. The league was founded by the Board of Control for Cricket in India (BCCI) in 2008, and is regarded as the brainchild of Lalit Modi, the founder and former commissioner of the league. IPL has an exclusive window in ICC Future Tours Programme.
The IPL is the most-attended cricket league in the world and in 2014 ranked sixth by average attendance among all sports leagues. In 2010, the IPL became the first sporting event in the world to be broadcast live on YouTube. The brand value of IPL in 2018 was US Dollar 6.3 billion, according to Duff & Phelps. According to BCCI, the 2015 IPL season contributed ₹11.5 billion (US Dollar 182 million) to the GDP of the Indian economy.
There have been eleven seasons of the IPL tournament. The current IPL title holders are the Chennai Super Kings, who won the 2018 season. The most successful franchises in the tournament are the Chennai Super Kings and Mumbai Indians with 3 tournament wins each.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/matches.csv')

In [ ]:
#Selecting top 3 rows from the dataframe
df.head(3)

**Let's check then number of observations in the data.**

In [ ]:
# Fetching number of rows in the dataframe
df.shape[0]

**Let's check the number of matches each team won till now in the IPL history.**

In [ ]:
df['winner'].sort_values().value_counts()

**Did you notice that we have "Rising Pune Supergiant" 2 times in the above list. One with the name as "Rising Pune Supergiant" and other as "Rising Pune Supergiants". Let's cleanse the data by modifying the "Rising Pune Supergiants" to "Rising Pune Supergiant". Also, it can be present in other columns as well like 'team1', 'team2' , 'toss_winner'. Lets replace in all the columns at once. Though, it can be done in many ways, we will use the apply() function to do so.**

In [ ]:
df.loc[df['winner'] == 'Rising Pune Supergiants', 'winner'] = 'Rising Pune Supergiant'
#But we have to replace in other series as well like 'team1', 'team2',etc.
df[['team1','team2','toss_winner','winner']] = df[['team1','team2','toss_winner','winner']].apply(lambda val : val.str.replace('Rising Pune Supergiants','Rising Pune Supergiant') )

**Let's now find the numbers of wins for each team.**

In [ ]:
df['winner'].sort_values().value_counts()

**Another way of doing the same as above, because we have id as primary key (meaning we have different id for each match).**

In [ ]:
# Grouping the number of matched won by teams and sorting it based on the count of the matches.
winners_df = df.groupby('winner', sort = False).count().id.sort_values(ascending = False)
winners_df

**Let's plot a simple bar graph for the above data.**

In [ ]:
# A simple plot for the winners.
winners_df.plot(kind = 'bar', title = 'Number of matches won in IPL', label = 'Number of wins' )

**Let's do some analysis on relation between winning the toss and the match. Let's find total number of times/matches where a team won the toss and also won the match. Also, we will find the opposite as well. Number of times team won the toss and lost the match.**

In [ ]:
# case where team won the toss and also won the match
winning_toss_winning_match = df[(df['toss_winner'] == df['winner'])]

In [ ]:
# cases where team won the toss but lost the match
winning_toss_losing_match = df[(df['toss_winner'] != df['winner'])]

In [ ]:
new_df = pd.DataFrame({"Team won the toss":[ winning_toss_winning_match.shape[0],winning_toss_losing_match.shape[0]  ]}, index = ['Won match', 'Lost match']  )

In [ ]:
#Let's plot the above obervations
new_df.plot( kind = 'bar' , legend = False , title = "Number of time a team won the toss and the match result of that match")

**From above analysis, we can say that there is no significant impact of toss on the match result.**

**Now, there are discussion on team winning the toss and choosing to bat and bowl first. We will analyse which had more success with winning the toss and batting first and which team had more success when they choose to bowl first.**

In [ ]:
# Same as above, now we will check how many times team winning the toss and  batting first won and how many times team bowling first won
# For this we have to check if the team won the toss and toss_decision is bat/field and winning team is toss_winner
# NOTE : in this type of analysis we are considering that the team has won the toss and its their decision to select to bat or bowl
team_batting_first_won = df.loc[(df['toss_winner'] == df['winner']) & (df['toss_decision'] == 'bat'), ['id', 'winner']]
team_fielding_first_won = df.loc[(df['toss_winner'] == df['winner']) & (df['toss_decision'] == 'field'),  ['id', 'winner']]

In [ ]:
team_fielding_first_won_df= team_fielding_first_won['winner'].value_counts()
team_batting_first_won_df = team_batting_first_won['winner'].value_counts()

In [ ]:
team_batting_first_won_df.plot(kind = 'bar', title = 'Team winning toss, batting first and winning the match.')

In [ ]:
team_fielding_first_won_df.plot(kind = 'bar', title = 'Team winning toss, fielding first won the match. ')

**Let's do another analysis**

In [ ]:
df['city'].value_counts()

**If you see above data, we have Bangalore and Bengaluru which refers to the same city.So we will replace "Banglaore" which is the old name of the city "Bengaluru".**

In [ ]:
#Two ways to do the same
df.loc[df['city'] == 'Bangalore', 'city'] = 'Bengaluru'
#Another way 
#Changing city name from Bangalore to Bengaluru
df['city'] = df['city'].str.replace('Bangalore','Bengaluru')

**Let's analyse how "Duckworth–Lewis method" affects result of match based on team batting first or bowling first.**

In [ ]:
#Creating a dataframe, which holds the records where the match result is decided by "Duckworth-Lewis method".
new_df = df.loc[ df['dl_applied'] != 0 ,['id','team1','team2','toss_winner','toss_decision','dl_applied','winner']]

In [ ]:
# Now we have to create a new column to decide which team has batted first for each match. I am gonna create a function for it and apply 
# that function to the dataframe ***

def set_batting_first(team1, team2, toss_winner, toss_decision):
    if toss_decision == 'bat':
        return toss_winner
    else:
        if team1 == toss_winner:
            return team2
        else:
            return team1  

In [ ]:
for items in new_df.loc[: , ['team1','team2','toss_winner','toss_decision','bat_first']].itertuples():
    new_df.loc[items[0] , 'bat_first'] = set_batting_first(items[1], items[2], items[3], items[4])

In [ ]:
#Now we are gonna create 1 more column named "is_bat_first_win" which will have 1 if the team batting first won and 0 if team batting 1 lost.
new_df['is_bat_first_win'] = 0
new_df.loc[new_df['bat_first'] == new_df['winner'],'is_bat_first_win'] = 1

In [ ]:
#Lets create a new dataframe with just two results as below.
dl_bat_first_win = pd.DataFrame(new_df['is_bat_first_win'].value_counts())
dl_bat_first_win.index = ['field_first_wins','bat_first_wins']

**Below figure shows the number of times "Duckworth–Lewis method" is applied and the team fielding first(or team batting second) won the match. This provided a direct connection between how "Duckworth–Lewis method" favours team batting second, though the topic is still argued the most.**

In [ ]:
dl_bat_first_win.plot(kind = 'bar')

**A win margin of more than 100 runs or more than 8 wickets in T20 is considered huge. Let's do an analysis on how many occassion, a team has lost the match with more than 100 run or more than 8 tickets. NOTE : we are gonna ignore the matches in which "Duckworth–Lewis method" is applied.**

In [ ]:
# New dataframe where DL method is not applied and win margin is 100 or more runs or more than 8 wickets.
new_df = df.loc[ (df['dl_applied'] == 0) & ((df['win_by_runs'] > 99) | (df['win_by_wickets'] > 8)) , ['id','team1','team2','winner','win_by_runs','win_by_wickets']]

In [ ]:
#creating a new column to track the losing team 
new_df['losing_team'] = np.nan

In [ ]:
# I will create a method to fetch the losing team from the dataframe
def check_losing_team(row):
    if row['team1'] == row['winner']:
        row['losing_team'] = row['team2']
        return row['team2']
    else:
        row['losing_team'] = row['team1']
        return row['team1']

In [ ]:
new_df['losing_team'] = new_df.apply(check_losing_team, axis = 1)

In [ ]:
new_df.losing_team.value_counts().plot(kind = 'bar', title = 'Number of times team losing with big margin')

**Let's do analysis on Number of times a player got the Man of the Match till 2017 edition of IPL.**

In [ ]:
# number of times the playes got Man of the match
mom_players = df.player_of_match.value_counts()

In [ ]:
# we want to analyse only those players who got the award atleast 10 times
mom_players[mom_players >= 10 ].plot(kind = 'bar', title = 'Number of times a player got Man of the Match')